First, open an ssh tunnel to the CatSim database hosted by the University of Washington.  Open a terminal window and type

```
ssh -L 51433:fatboy.phys.washington.edu:1433 simsuser@gateway.astro.washington.edu
```

There is some configuration that you will have to do to make sure this works.  Instructions are here:

https://confluence.lsstcorp.org/display/SIM/Accessing+the+UW+CATSIM+Database

In [ ]:
from __future__ import with_statement
import numpy as np

In [ ]:
import os
opsimdb = os.path.join("/Users","danielsf","physics")
opsimdb = os.path.join(opsimdb, "lsst_150412", "Development", "garage")
opsimdb = os.path.join(opsimdb, "OpSimData", "kraken_1042_sqlite.db")

In [ ]:
from lsst.sims.catUtils.baseCatalogModels import RRLyStarObj
stardb = RRLyStarObj()

In [ ]:
from lsst.sims.catUtils.utils import StellarLightCurveGenerator
lc_gen = StellarLightCurveGenerator(stardb, opsimdb)

In [ ]:
raRange = (60.0, 65.0)
decRange = (-15.0, -10.0)
bandpass = ('g', 'r', 'i')
pointings = lc_gen.get_pointings(raRange, decRange, bandpass=bandpass)

In [ ]:
print len(pointings)

In [ ]:
lc_dict, truth_dict = lc_gen.light_curves_from_pointings(pointings)

In [ ]:
lc_dict.keys()[-1]

In [ ]:
lc = lc_dict[853673991]

In [ ]:
print len(lc['g']['mjd']), len(lc['r']['mjd']), len(lc['i']['mjd'])

In [ ]:
print truth_dict[853673991]

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
fig, ax = plt.subplots()

ax.errorbar(lc['r']['mjd'], lc['r']['mag'], lc['r']['error'],
            fmt='', linestyle='None')
ax.set(xlabel='MJD', ylabel='r-band magnitude')

In [ ]:
from gatspy.periodic import LombScargleFast
model = LombScargleFast().fit(lc['r']['mjd'], lc['r']['mag'],
                              lc['r']['error'])
periods, power = model.periodogram_auto(nyquist_factor=100)

fig, ax = plt.subplots()
ax.plot(periods, power)
ax.set(xlim=(0.0001, 1.0))

In [ ]:
type(model)


In [ ]:
model.optimizer.period_range = (0.1, 1.4)
best_period = model.best_period
print best_period

In [ ]:
from lsst.utils import getPackageDir
import time

t_start = time.time()

reference_dict = {}
truth_dir = os.path.join(getPackageDir('sims_sed_library'), 'rrly_lc')

rrab_dir = os.path.join(truth_dir, 'RRab')
rrc_dir = os.path.join(truth_dir, 'RRc')

for file_name in os.listdir(rrab_dir):
    if 'per' in file_name:
        full_name = os.path.join(rrab_dir, file_name)
        with open(full_name, "r") as input_file:
            for line in input_file:
                vv = line.split()
                if vv[1] == 'Period':
                    reference_dict['rrly_lc/RRab/'+file_name] = float(vv[3])
                    break

for file_name in os.listdir(rrc_dir):
    if 'per' in file_name:
        full_name = os.path.join(rrc_dir, file_name)
        with open(full_name, "r") as input_file:
            for line in input_file:
                vv = line.split()
                if vv[1] == 'Period':
                    reference_dict['rrly_lc/RRc/'+file_name] = float(vv[3])
                    break


In [ ]:
import json

true_period = []
found_period = []

for kk in lc_dict.keys()[:10]:
    lc_obj = lc_dict[kk]
    truth = json.loads(truth_dict[kk])
    file_name = truth['pars']['filename']
    for ff in lc_obj.keys()[:1]:
        lc = lc_obj[ff]

        model = LombScargleFast().fit(lc['mjd'], lc['mag'],
                                      lc['error'])
        model.optimizer.period_range = (0.1, 1.4)
        best_period = model.best_period
        true_period.append(reference_dict[file_name])
        found_period.append(best_period)

true_period = np.array(true_period)
found_peroid = np.array(found_period)

In [ ]:
for tt, ff in zip(true_period, found_period):
    print tt, ff

# Generating your own cadence

In [ ]:
import numpy as np
from lsst.sims.catalogs.generation.db import fileDBObject

In [ ]:
dtype = np.dtype([('fieldRA', np.float), ('fieldDec', np.float),
                 ('expMJD', np.float), ('filter', str, 1),
                 ('FWHMeff', np.float), ('fiveSigmaDepth', np.float),
                 ('filtSkyBrightness', np.float)])


In [ ]:
from lsst.sims.utils import ObservationMetaData, solarRaDec
from lsst.sims.utils import altAzPaFromRaDec

import warnings

warnings.filterwarnings('ignore',message=".*UT1-UTC.*")

rng = np.random.RandomState(87)

field_ra = 112.0
field_dec = -31.0

mjd_candidates = rng.random_sample(1000)*3653.0+59580.0
mjd_candidates.sort()

obs_list = [ObservationMetaData(mjd=mm) for mm in mjd_candidates]
    
solar_alt = []
solar_az = []
for obs in obs_list:
    rr, dd = solarRaDec(obs.mjd.TDB)
    alt, az, pp = altAzPaFromRaDec(rr, dd, obs)
    solar_alt.append(alt)
    solar_az.append(az)  

print "getting solar ra dec took ",time.time()-t_start

solar_alt = np.array(solar_alt)
solar_az = np.array(solar_az)

dexes_sundown = np.where(solar_alt>10.0)
print len(dexes_sundown[0])

n_time = len(dexes_sundown[0])
    
obs_list = np.array(obs_list)[dexes_sundown]
    
alt_list = []
az_list = []
for obs in obs_list:
    alt, az, pp = altAzPaFromRaDec(field_ra, field_dec, obs)
    alt_list.append(alt)
    az_list.append(az)

alt_list = np.array(alt_list)
dexes = np.where(alt_list>45.0)

mjd_list = np.array([obs.mjd.TAI for obs in obs_list[dexes]])
airmass_list = np.array([1.0/np.cos(0.5*np.pi-alt)
                         for alt in np.radians(alt_list[dexes])])

In [ ]:
import time
t_start = time.time()
from lsst.utils import getPackageDir
from lsst.sims.photUtils import BandpassDict
atmos_dir = os.path.join(getPackageDir('throughputs'), 'atmos')
bp_dict = {}
hw_dict = {}
for airmass in np.arange(1.0, 2.6, 0.1):
    am_int = int(10*airmass)
    atmos_name = os.path.join(atmos_dir, 'atmos_%d.dat' % am_int)
    bp, hw = \
    BandpassDict.loadBandpassesFromFiles(atmoTransmission=atmos_name)
    bp_dict[am_int] = bp
    hw_dict[am_int] = hw
print "that took ",time.time()-t_start

In [ ]:
fwhm_eff_list = rng.random_sample(len(mjd_list))*0.1+0.65

In [ ]:
t_start = time.time()

import warnings
warnings.filterwarnings('ignore', message='.*UT1-UTC.*')

from lsst.sims.skybrightness import SkyModel
model = SkyModel(lowerAtm=True, upperAtm=True, scatteredStar=True)

from lsst.sims.utils import altAzPaFromRaDec
from lsst.sims.photUtils import Sed, calcM5, PhotometricParameters

bp_name_options = np.array(['g', 'r', 'i'])
bp_name_list = bp_name_options[rng.random_integers(0,2,
                                len(mjd_list))]
photParams = PhotometricParameters()

m5_list = []
sky_brightness_list = []

for mjd, airmass, fwhm, bp_name in \
    zip(mjd_list, airmass_list,
        fwhm_eff_list, bp_name_list):
        
    model.setRaDecMjd(field_ra, field_dec, mjd, degrees=True)
    wv, fl = model.returnWaveSpec()
    ss = Sed(wavelen=wv, flambda=fl[0])
    am_int = int(10*airmass)
    m5 = calcM5(ss,
                bp_dict[am_int][bp_name],
                hw_dict[am_int][bp_name],
                photParams, FWHMeff=fwhm)
    m5_list.append(m5)
    sky_brightness_list.append(ss.calcMag(bp_dict[am_int][bp_name]))

print "that took ",time.time()-t_start

In [ ]:
m5_list = np.array(m5_list)
sky_brightness_list = np.array(sky_brightness_list)
print sky_brightness_list.min()
print sky_brightness_list.max()
print field_ra
print field_dec

In [ ]:
import sqlite3
db_file_name = "example_test_cadence.db"
if os.path.exists(db_file_name):
    os.unlink(db_file_name)
conn = sqlite3.connect(db_file_name)
cc = conn.cursor()
cc.execute('''CREATE TABLE Summary
              (fieldRA float, fieldDec float, expMJD float,
              filter text, FWHMeff float, fiveSigmaDepth float,
              filtSkyBrightness float)''')

for mjd, bp, fwhm, m5, bright in \
zip(mjd_list, bp_name_list, fwhm_eff_list, m5_list, sky_brightness_list):
    cmd = '''INSERT INTO Summary VALUES(%.5f, %.5f, %.3f, '%s', %.5f, %.5f, %.5f)''' \
    % (np.radians(field_ra), np.radians(field_dec), mjd, bp, fwhm, m5,
      bright)
    cc.execute(cmd)
conn.commit()
conn.close()

In [ ]:
gen = StellarLightCurveGenerator(stardb, db_file_name)

In [ ]:
pointings = gen.get_pointings((field_ra-1.0, field_ra+1.0),
                              (field_dec-1.0, field_dec+1.0),
                              bandpass=('g','r','i'))

In [ ]:
len(pointings)

In [ ]:
len(pointings[0])

In [ ]:
lc, truth = gen.light_curves_from_pointings(pointings)

In [ ]:
len(lc)